<a href="https://colab.research.google.com/github/raokiey/foss4g_2023_japan_at_fukui_general_presentation_8/blob/main/segment_lidar_googlecolab_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segment Anything Modelを3次元点群データに適用してみた例

## このノートブックについて  
FOSS4G 2023 Japan@FUKUI コアデイ 一般発表8「Segment Anything Modelを
用いて地物抽出を試してみた」のスライド中に掲載している例を作成するために使用したノートブックです。  
このノートブックでは、3次元点群データに対して判読を行っています。  
`point_cloud_path`変数に判読したい3次元点群データ（lasもしくはlaz形式）のパスを設定することで、他のデータに対しても実行できます。  

## セットアップ  
Segment Anything Modelを3次元点群データに対して動かせるように環境構築などを行っています。  
使用したライブラリは以下の通りです。  
- segment-lidar
- cloth-simulation-filter

In [ ]:
!pip install cloth-simulation-filter
!pip install segment-lidar

In [ ]:
# サンプルで使用する3次元点群データのダウンロード
!wget -O ./sample_data/08NE3821.laz https://raokiey.github.io/foss4g_2023_japan_at_fukui_general_presentation_8/data/08NE3821.laz

In [ ]:
from segment_lidar import samlidar

## 3次元点群データへの適用例

サンプルの3次元点群データ（laz形式）として、以下のものを使用しています。  
- 静岡県「[VIRTUAL SHIZUOKA 静岡県 富士山南東部・伊豆東部 点群データ LPデータ　ダウンロードページ](https://www.geospatial.jp/ckan/dataset/shizuoka-2019-pointcloud/resource/d5e98a7b-f15c-45b0-bf40-0287f5b1de68)」  
08NE3821

In [ ]:
# segment-lidarを初期化、使用するSAMの重みをロード
model = samlidar.SamLidar(ckpt_path='sam_vit_h_4b8939.pth')

In [ ]:
# laz形式の3次元点群データをモデルに読み込む
point_cloud_path = './sample_data/08NE3821.laz'
points = model.read(point_cloud_path)

In [ ]:
 #CSF(Cloth Simulation Filter)アルゴリズムを用いて、地面をフィルタリング
cloud, non_ground, ground = model.csf(points)

In [ ]:
# 3次元点群データに対して、セグメンテーションを実行
labels, *_ = model.segment(points=cloud, image_path="raster.tif", labels_path="labeled.tif")

In [ ]:
# 判読結果を指定のパスに保存
save_path = './08NE3821_segmented.las'
model.write(points=points, non_ground=non_ground, ground=ground, segment_ids=labels, save_path=save_path)

## セグメンテーション結果の表示
`lted point`という名前のスカラーフィールドに結果は格納されています。  
セグメンテーション結果の可視化は[CloudCompare](https://www.danielgm.net/cc/)というソフトウェアをsegment-lidarのドキュメントで推奨しています。